In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
english_reviews_df = pd.read_json('preprocessed_english_reviews.json')
english_reviews_df.head()

,comments,comments_cleaned,comments_l,comments_token,comments_token_str,date,id,index,language,listing_id,probability,review_length,reviewer_id,reviewer_name,sentence_length
0,Daniel is really cool. The place was nice and ...,daniel really cool place nice clean quiet neig...,daniel is really cool. the place was nice and ...,"[daniel, realli, cool, place, nice, clean, qui...",daniel realli cool place nice clean quiet neig...,2009-03-30,1191,0,en,2818,0.992321,250,10952,Lam,46
1,Daniel is the most amazing host! His place is ...,daniel amazing host place extremely clean prov...,daniel is the most amazing host! his place is ...,"[daniel, amaz, host, place, extrem, clean, pro...",daniel amaz host place extrem clean provid eve...,2009-04-24,1771,1,en,2818,0.991255,334,12798,Alice,58
2,We had such a great time in Amsterdam. Daniel ...,great time amsterdam daniel excellent host fri...,we had such a great time in amsterdam. daniel ...,"[great, time, amsterdam, daniel, excel, host, ...",great time amsterdam daniel excel host friend ...,2009-05-03,1989,2,en,2818,0.995635,400,11869,Natalja,66
3,Very professional operation. Room is very clea...,professional operation room clean comfortable ...,very professional operation. room is very clea...,"[profession, oper, room, clean, comfort, locat...",profession oper room clean comfort locat close...,2009-05-18,2797,3,en,2818,0.990710,203,14064,Enrique,38
4,Daniel is highly recommended. He provided all...,daniel highly recommended provided necessities...,daniel is highly recommended. he provided all...,"[daniel, high, recommend, provid, necess, actu...",daniel high recommend provid necess actual wen...,2009-05-25,3151,4,en,2818,0.967161,277,17977,Sherwin,52


In [4]:
import pickle

def dummy_fun(token):
    return token

with open("ams_tfidf_vectorizer.pkl", "rb") as pklfile:
    count_vectorizer = pickle.load(pklfile)

In [12]:
automated_message = english_reviews_df[english_reviews_df.comments.str.contains("The host canceled this reservation")]
english_reviews_df.shape, automated_message.shape

((362300, 15), (7004, 15))

In [13]:
removed_automated_message = english_reviews_df[~english_reviews_df.comments.str.contains("The host canceled this reservation")]

In [8]:
count_vectorizer.get_feature_names()[:5]

['abl walk',
 'absolut amaz',
 'absolut love',
 'absolut perfect',
 'absolut recommend']

In [14]:
bag_of_words = count_vectorizer.transform(removed_automated_message['comments_token_str'])

In [15]:
dense_bow = bag_of_words.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((dense_bow > 0).sum()/dense_bow.size)*100, "%")

Sparsicity:  0.6615934319553274 %


In [16]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

# Tweak the two parameters below
number_topics = 6
number_words = 10

# Create and fit the LDA model
lda = LDA(n_components=number_topics, random_state=19, n_jobs=-1, learning_method='online')
lda.fit(bag_of_words)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=6, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=19, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [19]:
print_topics(lda, count_vectorizer, number_words)


Topic #0:
great locat locat great apart great great host place great high recommend host friend place nice easi walk wonder host

Topic #1:
good locat walk distanc minut walk apart clean citi center everyth need central station citi centr within walk comfort bed

Topic #2:
great place nice place place stay realli nice locat close well locat nice apart friend host recommend anyon close tram

Topic #3:
would definit enjoy stay realli enjoy definit recommend apart nice love stay definit stay would recommend public transport host great

Topic #4:
come back nice stay apart perfect felt like locat near nice quiet love host like home walk away amaz place

Topic #5:
high recommend love place place perfect everyth perfect great time nice host close citi citi center would high realli good


In [17]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda.score(bag_of_words))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda.perplexity(bag_of_words))

Log Likelihood:  -6302786.0884581795
Perplexity:  1764.2199118461087


In [18]:
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))